# 拡張思考のための効果的なプロンプト手法

このノートブックでは、Claude 3.7 Sonnet の拡張思考能力の有効性を最大限に高めるための高度なプロンプト手法について説明します。拡張思考をいつどのように使用するかについての以前のレッスンで学んだ内容を基に、最良の結果を得るためのプロンプトの *作成方法* に焦点を当てます。

このレッスンでは、次の内容を学びます:

1. 一般的な指示パターンとステップバイステップの指示パターンの違い
2. 拡張思考で N ショット プロンプトを実装する方法
3. 推論の深さと焦点を制御する手法
4. さまざまなユース ケースで再利用可能なプロンプト テンプレートを作成する方法

このノートブックの最後までに、Claude の拡張思考能力を最大限に活用するプロンプトを作成するための実用的な戦略を習得できます。

## セットアップ

まず、ユーティリティ関数をインポートし、Claude 3.7 Sonnet で動作するように Bedrock クライアントをセットアップします。

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Import our utility functions from previous lessons
import claude_utils

# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Claude 3.7 Sonnet model ID
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)

## 1. 一般的な指示パターンと段階的な指示パターン

クロードの拡張思考能力を扱う場合、指示を与えるための主なアプローチは 2 つあります。

1. **一般的な指示**: クロードが独自の思考アプローチを自由に決定できる、幅広く自由なガイダンス
2. **段階的な指示**: 特定の思考プロセスを概説する、規範的で連続的なガイダンス

それぞれのアプローチには、長所と理想的な使用例があります。両方の戦略を検討し、クロードの推論プロセスにどのような影響を与えるかを見てみましょう。

### 一般的な指示アプローチ

Claude の拡張思考機能を使用する場合、**一般的な指示は、規定的なステップバイステップのガイダンスよりも優れた結果をもたらすことが多い** です。これは最初は直感に反するように思えるかもしれませんが、拡張思考の仕組みを考えると納得できます。

![一般的な指示とステップバイステップの指示](./images/lesson4/general_vs_sbs.png)

拡張思考を有効にすると、Claude は問題を徹底的かつ系統的に解決する余裕ができます。正確な手順を規定するのではなく、「深く考える」ように一般的な指示を与えることで、Claude は独自の問題解決能力を活用できます。これは、人間が規定すると考える能力を超える可能性があります。

このアプローチが困難な問題でどのように機能するかを見てみましょう。

In [ ]:
# Define a problem to solve
problem = """
A retailer sells two products, Product A and Product B. 
Product A costs $20 to make and sells for $50. 
Product B costs $10 to make and sells for $25. 
The retailer has $2000 available for production costs, 
and warehouse space for at most 250 units in total.

Due to customer demand, they need to produce at least twice as many units of Product B as Product A.

What is the optimal production plan to maximize profit?
"""

# Define an effective general instruction that leverages extended thinking
general_instruction = """
Please think about this optimization problem thoroughly and in great detail.
Consider multiple approaches and show your complete reasoning.
"""

# Create the full prompt
full_prompt = f"{general_instruction}\n\n{problem}"

# Invoke Claude with extended thinking
response = claude_utils.invoke_claude(
    bedrock_runtime,
    full_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=4096,
    max_tokens=1200
)

# Display the response
claude_utils.display_claude_response(response)

### 一般的な指示が最も効果的である場合

上記の例は、拡張思考が有効になっている場合に、Claude が一般的な指示を使用して複雑な問題に効果的にアプローチする方法を示しています。このアプローチは、特に次の場合に効果的です。

- 複数の解決パスが存在する **オープンエンドの問題**
- 最適なアプローチがすぐにはわからない **複雑なシナリオ**
- 革新的な思考が価値のある **創造的な問題解決**
- さまざまな要素のバランスを取る必要がある **複数の制約のある問題**

Claude 3.7 で拡張思考を使用すると、通常、次の方法でより良い結果が得られます。

1. 期待される徹底性と深さについて大まかなガイダンスを提供する
2. 複数のアプローチの検討を促す
3. Claude が独自の思考プロセスを構造化できるようにする
4. 正確にどのように行うかではなく、何を達成するかに焦点を当てる

とはいえ、Claude は必要に応じて構造化された実行手順に効果的に従うことができます。このモデルは、以前のバージョンよりも複雑な指示を含むさらに長いリストを処理できます。

In [ ]:
# Let's create a helper function to demonstrate different instruction styles
def test_instruction_style(problem, instruction_style, reasoning_budget=4096):
    """
    Test a specific instruction style with Claude's extended thinking
    
    Args:
        problem (str): The problem to solve
        instruction_style (str): The instruction to prefix the prompt
        reasoning_budget (int): The reasoning budget to allocate
        
    Returns:
        dict: The full API response
    """
    full_prompt = f"{instruction_style}\n\n{problem}"
    
    print(f"Testing with instruction style: \n{instruction_style}\n")
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        full_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=1000
    )
    
    # Get just the first 300 characters of the response for a preview
    result_text = claude_utils.extract_response_content(response)
    preview = result_text[:300] + "..." if len(result_text) > 300 else result_text
    
    print("\nResponse preview:")
    print("-" * 80)
    print(preview)
    print("-" * 80)
    
    return response

In [ ]:
# Now let's try different instruction styles on a new problem

creative_problem = """
A city is experiencing increasing traffic congestion in its downtown area. 
The mayor has asked for innovative solutions that would reduce traffic by at least 20% 
while staying within a budget of $5 million and being implementable within 18 months.
What approach would you recommend?
"""

# First, test with a general, open-ended instruction
general_creative_instruction = """
Please think deeply and thoroughly about this urban planning challenge.
Consider multiple approaches, evaluate their pros and cons, and develop
a comprehensive recommendation.
"""

general_response = test_instruction_style(creative_problem, general_creative_instruction)

In [ ]:
# For comparison, let's try a more prescriptive approach
prescriptive_instruction = """
Analyze this urban planning problem using the following steps:
1. Identify at least 3 main causes of congestion
2. List exactly 5 potential solutions that fit the budget
3. Calculate the estimated traffic reduction for each solution
4. Create a table comparing all solutions by cost, timeline, and impact
5. Select the top solution and justify it
6. Outline an implementation plan with specific milestones
"""

prescriptive_response = test_instruction_style(creative_problem, prescriptive_instruction)

# Let's compare the token usage and timing for both approaches
general_tokens = general_response.get('usage', {}).get('totalTokens', 0)
general_time = general_response.get('_elapsed_time', 0)

prescriptive_tokens = prescriptive_response.get('usage', {}).get('totalTokens', 0)
prescriptive_time = prescriptive_response.get('_elapsed_time', 0)

print("\nComparison of approaches:")
print(f"General approach: {general_tokens} tokens in {general_time:.2f} seconds")
print(f"Prescriptive approach: {prescriptive_tokens} tokens in {prescriptive_time:.2f} seconds")

### 重要な洞察: バランスが重要

一般的に、一般的な指示は拡張思考に適していますが、ある程度の構造が役立つシナリオもあります。重要なのは、適切なバランスを見つけることです。

- **規範的すぎる**: クロードの思考を制限し、創造的な解決策を見逃す可能性があります
- **漠然としすぎる**: 集中的な問題解決のための十分なガイダンスを提供しない可能性があります

**ベスト プラクティス**: 拡張思考を使用する場合は、徹底的な推論を促す一般的な指示から始めます。構造的なガイダンスは、手元の特定のタスクに直接役立つ場合にのみ追加します。

規範的になりすぎずにある程度の構造を提供する、よりバランスの取れたアプローチを見てみましょう。

In [ ]:
# A balanced approach that provides some structure without being overly prescriptive
balanced_instruction = """
Please think deeply about this urban planning challenge and develop a comprehensive recommendation.

In your reasoning, consider:
- Various causes of downtown congestion
- A diverse range of potential solutions
- How each solution impacts different stakeholders
- Implementation feasibility within the constraints

Develop your thinking thoroughly before reaching a conclusion.
"""

balanced_response = test_instruction_style(creative_problem, balanced_instruction)

# Display full response for the balanced approach
print("\n\nFull response from balanced approach:")
claude_utils.display_claude_response(balanced_response)

## チェックイン ポイント: 一般的な指示と拡張思考

先に進む前に、理解を確認しましょう:

1. クロードの拡張思考能力を使用する場合、**一般的な指示** は通常、非常に規範的なステップごとのガイダンスよりも優れた結果をもたらします
2. 拡張思考により、クロードは問題を系統的に解決する余裕ができるので、**どのように** 行うかではなく、**何** を達成するかに集中できます
3. バランスの取れたアプローチは、クロードの問題解決能力を制限することなく、役立つコンテキストを提供します
4. クロードは必要に応じて構造化された手順に従うことができますが、過度に規範的なガイダンスは創造性を制限する可能性があります

では、次のテクニック、拡張思考による N ショット プロンプトに進みましょう。

## 2. 拡張思考による N ショット プロンプト

N ショット プロンプトは、特定の種類の問題へのアプローチ方法の例 (または「ショット」) を Claude に提供する強力な手法です。拡張思考と組み合わせると、このアプローチは Claude の推論プロセスをガイドすると同時に、深く考える自由を与えるのに特に効果的です。

### 拡張思考による N ショット プロンプトの仕組み

拡張思考による N ショット プロンプトを使用する場合、プロンプトに `<thinking>` や `<scratchpad>` などのタグを使用して推論パターンの例を含めることができます。Claude は次のことを行います。

1. 例の思考パターンを観察する
2. そのパターンを独自の拡張思考プロセスに一般化する
3. 同様の推論アプローチを新しい問題に適用する

このアプローチは、推論の *内容* を制約することなく推論の *スタイル* をガイドするのに役立ちます。

### N ショット プロンプトの実装

拡張思考による N ショット プロンプトの実装には、主に 2 つの方法があります:

1. **暗黙的な N ショット**: 推論と回答の両方を含む完全な例を提供する
2. **明示的な N ショット**: 推論プロセスを強調するために、明示的な `<thinking>` または同様のタグを含める

Claude で N ショット プロンプトを実装するためのヘルパー関数を作成しましょう:

In [ ]:
def create_nshot_prompt(examples, new_problem, thinking_tag="<thinking>"):
    """
    Create an N-shot prompt with reasoning examples
    
    Args:
        examples (list): List of dictionaries with 'problem', 'reasoning', and 'answer' keys
        new_problem (str): The new problem to solve
        thinking_tag (str): Tag to use for marking thinking sections
        
    Returns:
        str: Formatted N-shot prompt
    """
    prompt = "I'm going to show you some examples of how to solve problems, then I'll ask you to solve a new problem.\n\n"
    
    # Add the examples
    for i, example in enumerate(examples, 1):
        prompt += f"Example {i}:\n"
        prompt += f"Problem: {example['problem']}\n\n"
        prompt += f"{thinking_tag}\n{example['reasoning']}\n{thinking_tag}\n\n"
        prompt += f"Answer: {example['answer']}\n\n"
    
    # Add the new problem
    prompt += f"Now solve this problem:\n{new_problem}"
    
    return prompt

In [ ]:
# Create some examples with thinking patterns for N-shot prompting
examples = [
    {
        "problem": "If a train travels at 60 miles per hour, how far will it travel in 2.5 hours?",
        "reasoning": """
To solve this problem, I need to find the distance traveled given the speed and time.

The formula for distance is: distance = speed × time

Given:
- Speed = 60 miles per hour
- Time = 2.5 hours

Calculation:
distance = 60 miles/hour × 2.5 hours
distance = 150 miles
""",
        "answer": "The train will travel 150 miles in 2.5 hours."
    },
    {
        "problem": "A store is offering a 20% discount on an item that originally costs $85. What is the sale price?",
        "reasoning": """
To find the sale price, I need to calculate the discount amount and subtract it from the original price.

Given:
- Original price = $85
- Discount = 20%

Step 1: Calculate the discount amount.
Discount amount = Original price × Discount percentage
Discount amount = $85 × 20%
Discount amount = $85 × 0.20
Discount amount = $17

Step 2: Calculate the sale price.
Sale price = Original price - Discount amount
Sale price = $85 - $17
Sale price = $68
""",
        "answer": "The sale price is $68."
    }
]

# Create a new problem to solve using the N-shot pattern
new_math_problem = """
A car rental company charges $45 per day plus $0.25 per mile driven. 
If someone rents a car for 3 days and drives 420 miles, what is the total cost of the rental?
"""

# Create an N-shot prompt
nshot_prompt = create_nshot_prompt(examples, new_math_problem)

# Preview the prompt structure (first 500 characters)
print("N-shot prompt structure (excerpt):")
print("-" * 80)
print(nshot_prompt[:500] + "...")
print("-" * 80)

In [ ]:
# Test the N-shot prompt with extended thinking
nshot_response = claude_utils.invoke_claude(
    bedrock_runtime,
    nshot_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=4096,
    max_tokens=1000
)

# Display the result
claude_utils.display_claude_response(nshot_response)

### 拡張思考と N ショット プロンプトの威力

上記の例からわかるように、N ショット アプローチは、Claude の拡張思考プロセスを、提供された例と同様のパターンに従うように導きました。この手法は、次の場合に特に役立ちます。

1. **特定の推論構造が必要な場合**: 例によって、Claude は特定のアプローチを使用するように導かれます
2. **ドメイン固有の問題を解決している場合**: 例によって、ドメインの慣例や用語を示すことができます
3. **推論に焦点を合わせたい場合**: 例によって、Claude は問題の関連する側面に集中できます

N ショット プロンプトと拡張思考を組み合わせる利点は、*何を考えるか*を制約することなく、*どのように考えるか*に関するガイダンスを提供できることです。Claude は、新しい問題ごとに独自の側面に取り組む自由を持ち続けます。

推論パターンがより洗練された、より複雑な例を試してみましょう。

In [ ]:
# Create examples for a more complex reasoning pattern - business analysis
business_examples = [
    {
        "problem": "A SaaS company is experiencing a 5% monthly churn rate. What strategies could help reduce this churn?",
        "reasoning": """
To address the churn problem, I'll analyze potential causes and solutions through multiple lenses:

1) Customer Perspective Analysis:
- Customers may be churning due to: poor onboarding, missing features, bad support, or finding better alternatives
- Need to consider both new and longstanding customers who might churn for different reasons

2) Product-Market Fit Analysis:
- 5% monthly churn (60% annual) suggests potential product-market fit issues
- Need to identify if this affects all customer segments or just specific ones

3) Competitive Analysis:
- What are competitors offering that this company isn't?
- Where might the product be falling behind market standards?

4) Value Proposition Review:
- Is the product delivering on its promised value?
- Is pricing aligned with the perceived value?

Based on this analysis, my solution framework will address:
- Immediate tactical fixes to address acute churn issues
- Strategic improvements to enhance product-market fit
- Process changes to better monitor and prevent future churn
""",
        "answer": """
To reduce the 5% monthly churn rate, I recommend a three-pronged approach:

1. Immediate tactical improvements:
   - Implement exit surveys to identify specific pain points
   - Create a "save team" focused on retention for at-risk customers
   - Enhance onboarding and customer success touchpoints

2. Strategic product enhancements:
   - Develop a feature roadmap based on customer feedback
   - Consider pricing tier adjustments to better match value perception
   - Introduce loyalty incentives for long-term commitments

3. Process improvements:
   - Establish early warning systems to identify at-risk customers
   - Implement regular customer health scoring
   - Create cross-functional retention teams

These strategies should aim to bring churn down to industry benchmark levels (2-3% monthly) within two quarters.
"""
    }
]

# New business problem to solve
new_business_problem = """
An e-commerce company has a 68% shopping cart abandonment rate, which is hurting their conversion metrics.
What strategies would you recommend to reduce cart abandonment and increase completed purchases?
"""

# Create an N-shot prompt with the business example
business_nshot_prompt = create_nshot_prompt(business_examples, new_business_problem)

# Test the business N-shot prompt
business_response = claude_utils.invoke_claude(
    bedrock_runtime,
    business_nshot_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=8192,  # Using a larger budget for this complex problem
    max_tokens=1500
)

# Display the result
claude_utils.display_claude_response(business_response)

## N ショット パターンのバリエーション

上記の例は基本的な N ショット パターンを示していますが、ニーズに応じて使用できるバリエーションがいくつかあります:

1. **フォーマット ガイダンス付きのゼロ ショット**: 例の代わりに、タグを使用してフォーマット ガイドラインを提供します
2. **マルチステップ N ショット**: 複雑な多段階の推論プロセスを示す例
3. **比較 N ショット**: 複数のアプローチを比較する方法を示す例
4. **増分 N ショット**: 複雑さが増すにつれて互いに積み重なる例

### 拡張思考による N ショット プロンプトのベスト プラクティス

- **シンプルに始める**: 最小限の例から始め、必要な場合にのみ追加します
- **現実的な例を使用する**: 例を実際の問題に関連させます
- **複雑さに合わせる**: 例の複雑さは、対象の問題と同等にする必要があります
- **一貫したフォーマットを使用する**: すべての例で同じ構造を維持します
- **Claude に一般化させる**: 過度に複雑にならないようにします規範的。クロードにパターンを適応させましょう

次のテクニック、つまり推論の深さと範囲の制御に移りましょう。

## 3. 推論の深さと範囲を制御するテクニック

拡張思考により、クロードは推論する余地が広がりますが、推論を特定の方向に集中させたり、その深さを制御したりする必要がある場合があります。クロードの拡張思考プロセスを導く効果的なテクニックがいくつかあります。

1. **フォーカス質問**: 問題の特定の側面に注意を向ける
2. **制約仕様**: 境界と制限を明示的に示す
3. **深さインジケーター**: 特定の領域をどの程度徹底的に調査するかを指定する
4. **フレームワーク適用**: 特定の分析フレームワークを提案する

これらのテクニックは、従うべき正確な手順について過度に規定することなく、クロードの拡張思考を特定のニーズに合わせて調整するのに役立ちます。

In [ ]:
def test_reasoning_control(problem, control_technique, technique_name, reasoning_budget=4096):
    """
    Test a specific reasoning control technique
    
    Args:
        problem (str): The base problem to solve
        control_technique (str): The control technique to apply
        technique_name (str): Name of the technique for display
        reasoning_budget (int): The reasoning budget to allocate
        
    Returns:
        dict: The full API response
    """
    full_prompt = f"{control_technique}\n\n{problem}"
    
    print(f"\n=== Testing {technique_name} ===\n")
    print(f"Technique: {control_technique}\n")
    print(f"Problem: {problem}\n")
    
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        full_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=1200
    )
    
    # Display key metrics
    elapsed_time = response.get('_elapsed_time', 0)
    total_tokens = response.get('usage', {}).get('totalTokens', 0)
    output_tokens = response.get('usage', {}).get('outputTokens', 0)
    
    print(f"Time: {elapsed_time:.2f}s, Tokens: {total_tokens} (Output: {output_tokens})")
    
    return response

In [ ]:
# Base problem that we'll apply different control techniques to
base_problem = """
A retail company is planning to open a new store and needs to decide between three potential locations. 
The company wants to maximize profit while considering factors like foot traffic, rent costs, 
competition, and local demographics.

Location A: Downtown - High foot traffic, high rent ($12,000/month), high competition, affluent demographics
Location B: Shopping mall - Medium foot traffic, medium rent ($8,000/month), medium competition, mixed demographics
Location C: Suburban area - Low foot traffic, low rent ($5,000/month), low competition, family demographics
"""

# Test Focus Questions technique
focus_technique = """
Think deeply about this business decision problem. In your analysis, focus particularly on:

1. How does each location's foot traffic translate to potential sales volume?
2. What is the break-even point for each location given the different rent costs?
3. How might the competitive landscape affect market share at each location?
4. Which demographic groups are most likely to become high-value customers?
"""

focus_response = test_reasoning_control(
    base_problem, 
    focus_technique, 
    "Focus Questions Technique"
)

# Display a preview of the response
result_text = claude_utils.extract_response_content(focus_response)
preview = result_text[:300] + "..." if len(result_text) > 300 else result_text
print("\nResponse preview:")
print("-" * 80)
print(preview)
print("-" * 80)

In [ ]:
# Test Constraint Specification technique
constraint_technique = """
Please analyze this business location decision thoroughly. 

As you reason through this problem, consider the following constraints:
- The company needs to achieve profitability within 12 months
- Staffing costs will be roughly the same at all locations
- The company's brand appeals most strongly to millennials and young professionals
- The store requires at least 2,000 sq ft of floor space
- The company can invest no more than $150,000 in initial setup costs
"""

constraint_response = test_reasoning_control(
    base_problem, 
    constraint_technique, 
    "Constraint Specification Technique"
)

# Display a preview of the response
result_text = claude_utils.extract_response_content(constraint_response)
preview = result_text[:300] + "..." if len(result_text) > 300 else result_text
print("\nResponse preview:")
print("-" * 80)
print(preview)
print("-" * 80)

In [ ]:
# Test Framework Application technique
framework_technique = """
Please analyze this business location decision by applying a SWOT analysis 
(Strengths, Weaknesses, Opportunities, Threats) to each potential location.

In your thinking, first create a comprehensive SWOT analysis for each location, 
then use that analysis to make an informed recommendation.
"""

framework_response = test_reasoning_control(
    base_problem, 
    framework_technique, 
    "Framework Application Technique"
)

# Display the full response for this technique
print("\nFull response from Framework Application Technique:")
claude_utils.display_claude_response(framework_response)

### 推論の制御に関する重要な洞察

上記の例で示されているように、正確な手順について過度に規定することなく、Claude の推論を導く効果的な方法がいくつかあります。

1. **フォーカス質問** は、Claude の注意を、あなたが最も重要だと考える問題の特定の側面に向けます。これは、特定の側面を詳細に分析したい場合に便利です。

2. **制約仕様** は、Claude が考慮すべき境界と現実世界の制限を示します。これにより、分析が実際の現実に根ざし、非現実的な解決策が回避されます。

3. **フレームワーク アプリケーション** は、推論を構造化するための特定の分析ツールまたはフレームワークを提案します。これは、確立された分析アプローチが存在するドメイン固有の問題に特に役立ちます。

これらの各手法は、Claude が推論する *内容* をガイドしながら、推論に *どのように* アプローチするかを自由に決定できるようにします。このバランスは、拡張思考を最大限に活用するための鍵です。

## チェックイン ポイント: 推論の制御

先に進む前に、次のことを理解していることを確認してください:

1. 拡張思考のメリットを活用しながら、クロードさんの推論を導くことができます
2. さまざまな制御手法は、それぞれ異なる目的に役立ちます
3. 目標は、推論を過度に制限することなく、関連する側面に集中させることです
4. これらの手法を組み合わせることで、より微妙な制御を行うことができます

では、これらの手法を組み込んだ再利用可能なプロンプト テンプレートを作成する方法を見てみましょう。

## 4. 再利用可能なプロンプト テンプレートの作成

拡張思考のための効果的なプロンプト手法を開発したら、さまざまなユース ケースで再利用可能なテンプレートを作成できます。これらのテンプレートには次の利点があります。

1. タスクごとにカスタム プロンプトを作成する必要がなくなるため、時間が節約されます。
2. クロードが同様の問題に取り組む方法に一貫性が確保されます。
3. 最適な方法を学びながら、段階的に改良することができます。
4. チームがベスト プラクティスを採用しやすくなります。

拡張思考を効果的に活用するプロンプトを作成するためのシンプルなテンプレート システムを構築しましょう。

In [ ]:
class PromptTemplate:
    """
    A reusable template for creating prompts that work well with extended thinking
    """
    def __init__(self, template, name="Unnamed Template", description=""):
        """
        Initialize a prompt template
        
        Args:
            template (str): The template string with placeholders for variables
            name (str): A descriptive name for this template
            description (str): A description of what this template is designed for
        """
        self.template = template
        self.name = name
        self.description = description
    
    def format(self, **kwargs):
        """
        Format the template by filling in the placeholders
        
        Args:
            **kwargs: The variables to fill in the template
            
        Returns:
            str: The formatted prompt
        """
        try:
            return self.template.format(**kwargs)
        except KeyError as e:
            missing_key = str(e).strip("'")
            raise ValueError(f"Missing required variable: {missing_key}")
    
    def __str__(self):
        return f"PromptTemplate('{self.name}')"
    
    def get_info(self):
        """Get information about this template"""
        return {
            "name": self.name,
            "description": self.description,
            "required_variables": self._extract_variables()
        }
    
    def _extract_variables(self):
        """Extract variable names from the template string"""
        import re
        # Find all instances of {variable_name} in the template
        matches = re.findall(r'\{([^{}]+)\}', self.template)
        return list(set(matches))  # Return unique variable names

In [ ]:
# Create a collection of reusable templates for different use cases

# Business analysis template
business_analysis_template = PromptTemplate(
    name="Business Problem Analysis",
    description="Template for analyzing business problems with multiple dimensions",
    template="""
Please think deeply and methodically about the following business problem:

{problem_statement}

In your analysis, consider:
- The key stakeholders and their interests
- Short-term and long-term implications
- Financial and non-financial impacts
- Potential risks and mitigation strategies
- Implementation challenges

{additional_context}

Develop a comprehensive recommendation based on your analysis.
"""
)

# Technical problem solving template
technical_problem_template = PromptTemplate(
    name="Technical Problem Solver",
    description="Template for solving technical or mathematical problems",
    template="""
I need a thorough solution to the following technical problem:

{problem_statement}

Please think about this problem comprehensively. Consider:
- Different approaches to solving it
- Edge cases and potential issues
- Efficiency and optimization opportunities
- Implementation considerations

{constraints}

Explain your reasoning thoroughly before providing the final solution.
"""
)

# Creative brainstorming template
creative_template = PromptTemplate(
    name="Creative Brainstorming",
    description="Template for creative idea generation and development",
    template="""
I need creative ideas for the following scenario:

{scenario}

Please think expansively and consider unconventional approaches. In your thinking:
- Generate a wide range of possibilities
- Consider cross-domain inspiration
- Think about how to combine different elements in novel ways
- Evaluate the most promising ideas in more depth

{constraints}

Develop 3-5 of your best ideas into more detailed concepts.
"""
)

# List our available templates
templates = [business_analysis_template, technical_problem_template, creative_template]
for template in templates:
    info = template.get_info()
    print(f"Template: {info['name']}")
    print(f"Description: {info['description']}")
    print(f"Required variables: {', '.join(info['required_variables'])}")
    print()

In [ ]:
# Test our templates with real problems

# Business analysis example
business_prompt = business_analysis_template.format(
    problem_statement="""
Our e-commerce company is experiencing a high rate of cart abandonment (72%).
We need to understand why customers are abandoning their carts and how we can reduce this rate.
""",
    additional_context="""
Our target demographic is primarily 25-45 year olds.
We sell premium home goods with an average order value of $120.
Our main competitors have abandonment rates of around 60%.
"""
)

# Technical problem example
technical_prompt = technical_problem_template.format(
    problem_statement="""
Design a system to efficiently store and retrieve time-series data from IoT devices.
Each device sends temperature readings every 5 minutes, and we have 10,000 devices.
We need to query average temperatures across devices for any given time period.
""",
    constraints="""
The system should be able to handle at least 5 years of historical data.
Query response time should be under 1 second for common queries.
Storage costs should be minimized.
"""
)

# Creative problem example
creative_prompt = creative_template.format(
    scenario="""
A mid-sized city wants to increase community engagement with its public parks
while also addressing issues of sustainability and technological integration.
""",
    constraints="""
Proposals should be implementable within a budget of $500,000.
Solutions should appeal to diverse age groups and backgrounds.
Priority should be given to ideas that can be implemented within 12 months.
"""
)

# Pick one example to test with Claude
print("Testing business analysis template with Claude:")
print("-" * 80)
print(business_prompt)
print("-" * 80)

business_response = claude_utils.invoke_claude(
    bedrock_runtime,
    business_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=8192,
    max_tokens=1500
)

# Display the result
claude_utils.display_claude_response(business_response)

### テンプレート ライブラリの利点

示されているように、プロンプト テンプレートのライブラリを作成すると、いくつかの利点があります。

1. **一貫性**: 各テンプレートには、特定の種類のタスクのベスト プラクティスがカプセル化されています
2. **再利用性**: テンプレートは、同じ種類のさまざまな問題で再利用できます
3. **カスタマイズ**: テンプレートは、特定のシナリオのプロンプトをカスタマイズするための変数を受け入れます
4. **チームの調整**: テンプレートは、チームが同様の問題に対して一貫したアプローチを採用するのに役立ちます

テンプレート ライブラリを構築するときは、次の点に重点を置きます。

- **目的固有のテンプレート**: 一般的なテンプレートではなく、特定の種類のタスク用のテンプレートを作成します
- **明確な変数名**: プレースホルダーに説明的な名前を使用して、テンプレートを使いやすくします
- **思慮深い構造**: クロードの推論を制約することなく導くテンプレートを設計します
- **継続的な改善**: 生成される応答の品質に基づいてテンプレートを改良します

別のテンプレートをテストして、拡張思考でのパフォーマンスを確認しましょう。

In [ ]:
# Let's test the technical problem template
print("Testing technical problem template with Claude:")
print("-" * 80)
print(technical_prompt)
print("-" * 80)

technical_response = claude_utils.invoke_claude(
    bedrock_runtime,
    technical_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=8192,
    max_tokens=1500
)

# Display the result
claude_utils.display_claude_response(technical_response)

### 完全なテンプレート システムの構築

本番環境では、テンプレート システムを拡張して次の機能を追加したい場合があります。

1. **テンプレート検証**: テンプレートがベスト プラクティスに従っているかどうかをチェックする
2. **パフォーマンス追跡**: どのテンプレートが最良の結果を生み出すかを監視する
3. **バージョン管理**: テンプレートの変更を時間の経過とともに追跡する
4. **テンプレート共有**: チームが効果的なテンプレートを簡単に共有できるようにする

作成したシンプルな `PromptTemplate` クラスは、特定のユース ケースで必要に応じてこれらの機能を含めるように拡張できます。

パフォーマンス追跡を含めるようにシステムを拡張する方法の例を次に示します。

In [ ]:
class TrackedPromptTemplate(PromptTemplate):
    """
    An extended prompt template that tracks performance metrics
    """
    def __init__(self, template, name="Unnamed Template", description=""):
        super().__init__(template, name, description)
        self.usage_history = []
    
    def execute(self, client, model_id, variables, reasoning_budget=4096, max_tokens=1000):
        """
        Execute the template and track performance
        
        Args:
            client: The bedrock_runtime client
            model_id (str): The model ID to use
            variables (dict): Variables to fill in the template
            reasoning_budget (int): The reasoning budget to allocate
            max_tokens (int): Maximum tokens for the response
            
        Returns:
            dict: The full API response
        """
        # Format the prompt
        prompt = self.format(**variables)
        
        # Invoke Claude
        start_time = time.time()
        response = claude_utils.invoke_claude(
            client,
            prompt,
            model_id,
            enable_reasoning=True,
            reasoning_budget=reasoning_budget,
            max_tokens=max_tokens
        )
        elapsed_time = time.time() - start_time
        
        # Track performance
        metrics = {
            "timestamp": time.time(),
            "prompt_length": len(prompt),
            "variables": list(variables.keys()),
            "reasoning_budget": reasoning_budget,
            "elapsed_time": elapsed_time,
            "input_tokens": response.get('usage', {}).get('inputTokens', 0),
            "output_tokens": response.get('usage', {}).get('outputTokens', 0),
            "total_tokens": response.get('usage', {}).get('totalTokens', 0)
        }
        
        self.usage_history.append(metrics)
        
        return response
    
    def get_performance_summary(self):
        """Get a summary of performance metrics for this template"""
        if not self.usage_history:
            return "No usage history available"
        
        # Calculate averages
        avg_time = sum(m["elapsed_time"] for m in self.usage_history) / len(self.usage_history)
        avg_tokens = sum(m["total_tokens"] for m in self.usage_history) / len(self.usage_history)
        
        return {
            "name": self.name,
            "usage_count": len(self.usage_history),
            "avg_time": avg_time,
            "avg_tokens": avg_tokens,
            "last_used": max(m["timestamp"] for m in self.usage_history)
        }

# Create a tracked template
tracked_creative_template = TrackedPromptTemplate(
    name="Tracked Creative Brainstorming",
    description="Template for creative idea generation with performance tracking",
    template=creative_template.template
)

# Test the tracked template
tracked_response = tracked_creative_template.execute(
    bedrock_runtime,
    CLAUDE_37_SONNET_MODEL_ID,
    {
        "scenario": """
        A technology startup wants to create a new mobile app that encourages people 
        to reduce their carbon footprint through gamification.
        """,
        "constraints": """
        The app should be engaging enough to retain users for at least 3 months.
        The business model should be sustainable without requiring users to pay.
        The app should provide measurable environmental impact.
        """
    },
    reasoning_budget=6144
)

# Display performance metrics
print("Performance metrics:")
print(tracked_creative_template.get_performance_summary())

# Display a preview of the response
result_text = claude_utils.extract_response_content(tracked_response)
preview = result_text[:500] + "..." if len(result_text) > 500 else result_text
print("\nResponse preview:")
print("-" * 80)
print(preview)
print("-" * 80)

## 要約: 拡張思考のための効果的なプロンプト手法

このノートブックでは、Claude 3.7 Sonnet の拡張思考機能を最大限に活用するプロンプトを作成するための強力な手法をいくつか検討しました:

1. **一般的な手順とステップバイステップの手順**
- 拡張思考では、一般的な手順の方が効果的です
- 幅広いガイダンスを提供しながら、Claude が独自の推論パスを決定できるようにします
- 思考プロセスを過度に制約することなく、構造を提供するバランスを見つけます

2. **拡張思考による N ショット プロンプト**
- `<thinking>` などのタグを使用して推論パターンの例を提供します
- コンテンツを制約することなく、推論のスタイルをガイドします
- ターゲットの問題の複雑さに一致する現実的な例を使用します

3. **推論の深さと範囲の制御**
- 特定の側面に注意を向けるために焦点となる質問を使用します
- 分析を実際の現実に根付かせるために制約を指定します
- 推論プロセスを構造化するためにフレームワークを適用します

4. **再利用可能なプロンプトの作成テンプレート**
- さまざまなタイプのタスク用に目的別のテンプレートを開発する
- 特定のシナリオに合わせてプロンプトをカスタマイズするための変数を含める
- パフォーマンスを追跡してテンプレートを継続的に改善する

これらのテクニックを適用することで、Claude の拡張思考能力を最大限に活用したプロンプトを作成し、より徹底的で洞察力に富んだ価値ある応答を得ることができます。

## 次のステップ

拡張思考のための効果的なプロンプトを作成する方法がわかったので、次のことを検討してください。

1. **ツール使用の統合**: 拡張思考とツール呼び出しを組み合わせる (レッスン 5 で説明)

2. **長文コンテンツの生成**: 拡張思考を使用して包括的なコンテンツを生成する (レッスン 6)

3. **複雑な問題の解決**: 拡張思考を適用して、さまざまなドメインの困難な問題に取り組む (レッスン 7)

## 練習問題

学習した内容を強化するために、次の練習問題を試してください。

1. ドメインの複雑な問題に対する一般的な指示プロンプトを作成する
2. ユースケースに関連する例を使用して N ショット プロンプトを開発する
3. 実行する一般的なタスクの再利用可能なテンプレートを作成する
4. 同じ問題に対するさまざまなプロンプト手法の結果を比較する

効果的なプロンプトは芸術であると同時に科学でもあることを忘れないでください。継続的な実験と改良により、Claude の拡張された思考能力を活用して、一貫して高品質の結果を生み出すプロンプトを開発できるようになります。